In [1]:
import cv2 as cv
import numpy as np

def dense_optical_flow(video_path, output_path):
    # Video girişi oluştur
    cap = cv.VideoCapture(video_path)
    ret, first_frame = cap.read()
    
    if not ret:
        print("Video açılırken hata oluştu.")
        return

    # İlk kareyi griye dönüştürme
    prev_gray = cv.cvtColor(first_frame, cv.COLOR_BGR2GRAY)
    
    # Çıkış videosu oluşturma
    height, width, _ = first_frame.shape
    fourcc = cv.VideoWriter_fourcc(*'XVID')
    out = cv.VideoWriter(output_path, fourcc, cap.get(cv.CAP_PROP_FPS), (width, height))

    # Görselleştirme için maske oluşturma
    mask = np.zeros_like(first_frame)
    mask[..., 1] = 255  # Doygunluğu maksimum yaparak renkleri daha belirgin hale getiriyoruz

    while cap.isOpened():
        # Yeni kareyi okuma
        ret, frame = cap.read()
        if not ret:
            break

        # Kareyi griye dönüştürme
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        
        # Farneback yöntemiyle yoğun optical flow hesaplama
        flow = cv.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

        # Optical flow'un büyüklük ve yönünü hesaplama
        magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1])
        
        # Yönlere göre hue değeri belirleme
        mask[..., 0] = angle * 180 / np.pi / 2
        
        # Büyüklüğe göre value değeri belirleme
        mask[..., 2] = cv.normalize(magnitude, None, 0, 255, cv.NORM_MINMAX)
        
        # HSV'den RGB (BGR) renk alanına dönüştürme
        rgb = cv.cvtColor(mask, cv.COLOR_HSV2BGR)

        # Sonuç kareyi videoya yazma
        out.write(rgb)
        
        # Optical flow sonucu görüntüleme
        cv.imshow("Dense Optical Flow", rgb)

        # Çıkmak için 'q' tuşuna basma seçeneği
        if cv.waitKey(1) & 0xFF == ord('q'):
            break

        # Sonraki kareye geçerken önceki kareyi güncelleme
        prev_gray = gray

    # Kaynakları serbest bırakma
    cap.release()
    out.release()
    cv.destroyAllWindows()


import cv2 as cv
import numpy as np

def dense_optical_flow_with_arrows(video_path, output_path):
    # Video girişi oluştur
    cap = cv.VideoCapture(video_path)
    ret, first_frame = cap.read()
    
    if not ret:
        print("Video açılırken hata oluştu.")
        return

    # İlk kareyi griye dönüştürme
    prev_gray = cv.cvtColor(first_frame, cv.COLOR_BGR2GRAY)
    
    # Çıkış videosu oluşturma
    height, width, _ = first_frame.shape
    fourcc = cv.VideoWriter_fourcc(*'XVID')
    out = cv.VideoWriter(output_path, fourcc, cap.get(cv.CAP_PROP_FPS), (width, height))

    # Görselleştirme için maske oluşturma
    mask = np.zeros_like(first_frame)
    mask[..., 1] = 255  # Doygunluğu maksimum yaparak renkleri daha belirgin hale getiriyoruz

    while cap.isOpened():
        # Yeni kareyi okuma
        ret, frame = cap.read()
        if not ret:
            break

        # Kareyi griye dönüştürme
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        
        # Farneback yöntemiyle yoğun optical flow hesaplama
        flow = cv.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

        # Optical flow'un büyüklük ve yönünü hesaplama
        magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1])
        
        # Yönlere göre hue değeri belirleme
        mask[..., 0] = angle * 180 / np.pi / 2
        # Büyüklüğe göre value değeri belirleme
        mask[..., 2] = cv.normalize(magnitude, None, 0, 255, cv.NORM_MINMAX)
        
        # HSV'den RGB (BGR) renk alanına dönüştürme
        rgb = cv.cvtColor(mask, cv.COLOR_HSV2BGR)

        # Okları çizme
        step = 15  # Piksel aralığı (isteğe göre ayarlanabilir)
        for y in range(0, gray.shape[0], step):
            for x in range(0, gray.shape[1], step):
                # Akış vektörlerinin başlangıç ve bitiş noktalarını hesapla
                fx, fy = flow[y, x]
                end_x = int(x + fx)
                end_y = int(y + fy)
                # Optical flow yönünü okla göster
                cv.arrowedLine(rgb, (x, y), (end_x, end_y), (0, 255, 0), 1, tipLength=0.4)

        # Sonuç kareyi videoya yazma
        out.write(rgb)
        
        # Optical flow sonucu görüntüleme
        cv.imshow("Dense Optical Flow with Arrows", rgb)

        # Çıkmak için 'q' tuşuna basma seçeneği
        if cv.waitKey(1) & 0xFF == ord('q'):
            break

        # Sonraki kareye geçerken önceki kareyi güncelleme
        prev_gray = gray

    # Kaynakları serbest bırakma
    cap.release()
    out.release()
    cv.destroyAllWindows()

In [2]:

# Kullanım:
video_path = "data/highway_2.mp4"  # Girdi videosunun yolu
output_path = "data/highway_2_dense_optical_flow_with_arrows.mp4"  # Çıkış videosunun yolu
dense_optical_flow_with_arrows(video_path, output_path)

OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
2024-11-13 18:09:23.288 Python[23633:2799283] +[IMKClient subclass]: chose IMKClient_Modern
2024-11-13 18:09:23.288 Python[23633:2799283] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [1]:

# Kullanım:
video_path = "data/highway_2.mp4"  # Girdi videosunun yolu
output_path = "data/highway_2_dense_optical_flow.mp4"  # Çıkış videosunun yolu
dense_optical_flow(video_path, output_path)


OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
2024-11-13 18:00:05.376 Python[23620:2796262] +[IMKClient subclass]: chose IMKClient_Modern
2024-11-13 18:00:05.376 Python[23620:2796262] +[IMKInputSession subclass]: chose IMKInputSession_Modern
